# Read time comparison: Zarr vs HDF5 
Compute the maximum water level during Hurricane Ike on a 9 million node triangular mesh storm surge model (this reads 53GB of data). The data was stored in both Zarr format and as NetCDF4/HDF5, using 11MB chunks with no filters and zlib (level 5) compression.  

Instead of reading the NetCDF4/HDF5 file with an HDF5 or NetCDF4 library, we extract the metadata into an fsspec referenceFileSystem file, create a mapper, and then read the mapper using the Zarr library.

Using a cluster with 60 cores, we find that the performance between this read approach and reading native Zarr is not significantly different.  

In [ ]:
import xarray as xr
import zarr
import fsspec
import fsspec.implementations.reference as refs
import intake
import intake_xarray

### Open Intake Catalog

In [ ]:
cat = intake.open_catalog('intake_catalog.yml')
list(cat)

### Zarr library reading HDF5 file with fsspec

In [ ]:
ds_hdf5  = cat['ike-hdf5'].to_dask()
print(ds_hdf5.zeta.encoding,'\n')
ds_hdf5.zeta

### Zarr library reading equivalent Zarr format dataset

In [ ]:
ds_zarr  = cat['ike-zarr'].to_dask()
print(ds_zarr.zeta.encoding,'\n')
ds_zarr.zeta

### Start a dask cluster to crunch the data

In [ ]:
import sys, os
sys.path.append(os.path.join(os.environ['HOME'],'shared','users','lib'))
import ebdpy as ebd
aws_profile = 'esip-qhub'
ebd.set_credentials(profile=aws_profile)

aws_region = 'us-west-2'
endpoint = f's3.{aws_region}.amazonaws.com'
ebd.set_credentials(profile=aws_profile, region=aws_region, endpoint=endpoint)
worker_max = 30
client,cluster = ebd.start_dask_cluster(profile=aws_profile, worker_max=worker_max, 
                                      region=aws_region, use_existing_cluster=True,
                                      adaptive_scaling=False, wait_for_cluster=False, 
                                      worker_profile='Medium Worker', propagate_env=True)

In [ ]:
cluster.scale(30);

In [ ]:
cluster

In [ ]:
client

In [ ]:
%%time
max_hdf5 = ds_hdf5['zeta'].max(dim='time').compute()

In [ ]:
%%time
max_zarr = ds_zarr['zeta'].max(dim='time').compute()

### compare a point from both datasets, should be the same

In [ ]:
max_hdf5[1000].values

In [ ]:
max_zarr[1000].values